<h1>Team: QemSpin <h1>
<h1>Last updated: 8th October 2023.<h1>

#  Introduction to our work

<font size=5>
1: Circuit Optimization. The conventional Givens gate encompasses both single and double excitation gates, thereby ensuring the conservation of particle numbers, that is, the Hamming weight of the quantum state remains consistent(i.e. 9 for OH). However, this comprehensiveness also escalates the simulation cost substantially. To enhance efficiency, we optimized the conventional Givens gate by eliminating the minimal contributions from both single and double excitation states. <br>

2: Error mitigation. We used VQE to calculate the energy values of the three quantum circuits, and linearly fitted the obtained energy values, extrapolating them to the energy values without noise.

## 1 Circuit Optimization.
<font size=5>

* We implement it on the SpinQit platform. As the result, we optimized the number of single excited state gates from 13 to 1, and the number of double excited state gates from 46 to 1.

* Please refer to Chosed_Adapt_Givens_Ansatz for the results.

### 1.1 The optimized circuit.
<div align=left><img src="IMG/Optimized_Circuit.png" width=800 length=800></div>

## 1.2 The results of a circuit with and without noise.

In [ ]:
from datetime import datetime

import numpy as np
import time

from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper,ParityMapper,QubitConverter
from qiskit.algorithms.minimum_eigensolvers import VQE
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
from qiskit.algorithms.optimizers import *  


from Estimator_with_noise import estimator_noise
from Estimator_without_noise import estimator_expect
from Ansatz import Adapt_Givens_Ansatz
from Noise_models import fakecairo, fakekolkata, fakemontreal


In [ ]:
# Set up molecule
ultra_simplified_ala_string = """
O 0.0 0.0 0.0
H 0.45 -0.1525 -0.8454
"""
driver = PySCFDriver(
    atom=ultra_simplified_ala_string.strip(),
    basis='sto3g',
    charge=1,
    spin=0,
    unit=DistanceUnit.ANGSTROM
)
qmolecule = driver.run()
mapper = JordanWignerMapper()

In [ ]:
# Set up VQE
seed_pool = [20, 21, 30, 33, 36, 42, 43, 55, 67, 170]
noise_word = ['fakecairo','fakekolkata','fakemontreal']
ansatz = Adapt_Givens_Ansatz()
optimizer = COBYLA(maxiter=100)
ground_energy = -74.38714627
shots = 6000

## 1.2.1 Computed the ground energy without noise

In [ ]:
average_error_rate, average_times = 0, 0

seed = 20
    
# Callback to store intermediate results during optimization
parameters_list = []
values = []
last_call_time = None

def callback(eval_count, parameters, mean, std):
    global last_call_time
    current_call_time = datetime.now()
    if last_call_time is not None:
        print(f"Cost Time: {current_call_time - last_call_time}")

    last_call_time = current_call_time
    parameters_list.append(parameters)
    values.append(mean)
    print(f'iter: {len(parameters_list)}, loss: {mean}, params: {parameters}')        
    
estimator = estimator_expect(seed, shots)

vqe_solver = VQE(estimator, ansatz, optimizer, callback=callback)
vqe_solver.initial_point = [0.0] * ansatz.num_parameters

calc = GroundStateEigensolver(mapper, vqe_solver)

start_time = time.time()
res = calc.solve(qmolecule)
end_time = time.time()

cost_time = end_time - start_time
result = res.computed_energies + res.nuclear_repulsion_energy
error_rate = abs((ground_energy - result) / ground_energy * 100)

with open(f'Documents/Data_Without_Noise/Exact_expect_Adapt_Givens_COBYLA.txt'.format(),"a") as f:
    f.write(f'seeds = {seed},Time = {cost_time}, '
                f'energy = {res.computed_energies}, Error rate: {error_rate},parameters = {parameters_list[-1]} '
                f'optimizer = COBYLA, Ansatz = Adapt_Givens \n')

## 1.2.2 Computed the ground energy with noise.

In [ ]:
## Computed the ground energy with noise
for index, noise in enumerate(noise_word):
    noise_model = globals()[noise]()
    average_error_rate, average_times = 0, 0
    
    for seed in seed_pool:
        print(f'\n Seed = {seed}, Noise model = {noise}')


        # Callback to store intermediate results during optimization
        parameters_list = []
        values = []
        last_call_time = None


        def callback(eval_count, parameters, mean, std):
            global last_call_time
            current_call_time = datetime.now()
            if last_call_time is not None:
                print(f"Cost Time: {current_call_time - last_call_time}")

            last_call_time = current_call_time
            parameters_list.append(parameters)
            values.append(mean)
            print(f'iter: {len(parameters_list)}, loss: {mean}, params: {parameters}')        
            
        estimator = estimator_noise(seed, shots, noise_model)
        
        vqe_solver = VQE(estimator, ansatz, optimizer, callback=callback)
        vqe_solver.initial_point = [0.0] * ansatz.num_parameters
        
        calc = GroundStateEigensolver(mapper, vqe_solver)
        
        start_time = time.time()
        res = calc.solve(qmolecule)
        end_time = time.time()
        
        cost_time = end_time - start_time
        result = res.computed_energies + res.nuclear_repulsion_energy
        error_rate = abs((ground_energy - result) / ground_energy * 100)
        
        with open(f'Documents/Data_With_Noise/noise_{noise}_Adapt_Givens_COBYLA.txt'.format(),"a") as f:
            f.write(f'Noise_model: {noise}, seeds = {seed},Time = {cost_time}, '
                     f'energy = {res.computed_energies}, Error rate: {error_rate},parameters = {parameters_list[-1]} '
                     f'optimizer = COBYLA, Ansatz = Adapt_Givens \n')
            
        average_error_rate += error_rate
        average_times += cost_time
        
    # with open(f'Documents/noise_{noise}_Adapt_Givens_COBYLA.txt'.format(),"a") as f:
    #     f.write(f'Noise_model: {noise}, optimizer = COBYLA, Average_Time = {average_times / len(seed_pool)}, '
    #              f'Average Error rate: {average_error_rate/len(seed_pool)}\n')


## 2： Error mitigation.

### 2.1 We used VQE to calculate the energy values of the following three quantum circuits, and linearly fitted the obtained energy values, extrapolating them to the energy values without noise.
<font size=5>

* Scale one: 
<div align=left><img src="IMG/scale_one.png" width=800 length=800></div>

* Scale three: 
<div align=left><img src="IMG/scale_three.png" width=800 length=800></div>

* Scale five: 
<div align=left><img src="IMG/scale_five.png" width=800 length=800></div>

## 2.2 Data Fitting

In [ ]:
import numpy as np
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
noise_word = ['fakecairo','fakekolkata','fakemontreal']

for noise in noise_word:
    # 原始数据点
    x = np.array([1, 3, 5])
    y = np.load(f'Documents/Error_mitigation_data/{noise}_seed_20.npy')

    # 定义指数函数模型
    def model(x, a, b ):
        return a * np.exp(b * x)

    # 进行指数拟合
    popt, pcov = curve_fit(model, x, y)
    a, b  = popt

    # 打印拟合参数
    print(f"The coefficients are: a={a}, b={b}")

    value_at_zero = model(0, a, b)
    print(f'The energy with non-noise in {noise} model: ,{value_at_zero} \n')

    # 生成用于绘制拟合函数的数据点
    x_plot = np.linspace(0, 5, 1000)
    y_plot = model(x_plot, a, b)

    # 绘制原始数据点
    plt.plot(x, y, 'o', label='Original data')

    # 绘制拟合曲线
    plt.plot(x_plot, y_plot, 'r-', label='Fitted Curve')

    plt.title(f'Noise model: {noise}')
    plt.xlabel('Scale')
    plt.ylabel('Energy')
    plt.legend()
    plt.savefig(f'IMG/Data_Fitting/{noise}_seed_20.png')
    plt.show()

# Conclusion

- Average error rate under noise-free conditions and  with noise
<div align=left><img src="IMG/Average error rate under noise-free conditions and  with noise.png" width=800 length=800></div>

- Error rate with noise and under error mitigaion while seed is 20 
<div align=left><img src="IMG/Error rate  with noise and under mitigation while seed is 20.png" width=800 length=800></div>